In [1]:
"""This notebook is to optimize hyperparameters for LSTM Model Classification using Randomized Search"""

# seperated by npast_timesteps to make preprocessing easier (tried doing it inside the wrapper and ran into many shape issues) 
# kernal crashed when I tried to do it all in one for loop

'This notebook is to optimize hyperparameters for LSTM Model Classification using Randomized Search'

In [2]:
# imports 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nrsim_lstm import NRSIM_LSTM as lstm
from Archive import utils277b as utils

2024-12-12 13:39:48.662462: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-12 13:39:48.663123: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 13:39:48.665464: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-12 13:39:48.671673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734039588.682591   11611 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734039588.68

## Data Preprocessing and Set Parameters for LSTM

In [3]:
# load in data
data = pd.read_csv("data/Sim_all_casualties.csv")
data.head()

,Time,pH,Hydrogen,Total Gas,Temperature,Pressure,Radioactivity,Power,Reactor Safety,Injection of Air,Injection of Air Degree,Resin Overheat,Resin Overheat Degree,Fuel Element Failure,Fuel Element Failure Degree,Chemical Addition,Vent Gas
0,0,11.000,50.000000,60.000000,500.000000,2100.000000,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,1,10.998,50.112998,60.112998,500.732703,2104.971916,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,2,10.996,50.225941,60.225941,501.463398,2109.930204,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,3,10.994,50.338517,60.338517,502.190083,2114.861274,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,4,10.992,50.450414,60.450414,502.910764,2119.751611,10.0,100.0,0,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [4]:
numerical = ['Time', 'pH', 'Hydrogen', 'Total Gas', 'Temperature', 'Pressure','Radioactivity', 'Power']
categorical = ['Reactor Safety', 'Injection of Air', 'Injection of Air Degree', 
               'Resin Overheat', 'Resin Overheat Degree', 'Fuel Element Failure', 
               'Fuel Element Failure Degree', 'Chemical Addition', 'Vent Gas']


# In case there are NaN values
data = data.fillna(False)
data[categorical] = data[categorical].astype(int) # set categorical as integer

# make sure no NaN values
unique_values = data.apply(pd.Series.unique)
# print(unique_values)

/tmp/ipykernel_11611/1598807462.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(False)


In [5]:
# training matrices
X = np.array(data[numerical].drop(columns=['Time']))
Y = np.array(data[categorical])

## Randomized Search - Classification

In [6]:
from sklearn.base import BaseEstimator, ClassifierMixin
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

class PreprocessingWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, n_timesteps=1, n_features=7, neurons=[32, 32], activation='tanh', 
                 optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'],
                 dropout=0.2, conv_layer=False, nfilters=64, conv_act='relu', pool_size=2, 
                 classification=True, n_predicted_timesteps = 1, epochs = 50, batch = 64, n_predicted_features = 9):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.neurons = neurons
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss
        self.metrics = metrics
        self.dropout = dropout
        self.conv_layer = conv_layer
        self.nfilters = nfilters
        self.conv_act = conv_act
        self.pool_size = pool_size
        self.classification = classification
        self.n_predicted_timesteps = n_predicted_timesteps
        self.epochs = epochs
        self.batch  = batch
        self.n_predicted_features = n_predicted_features
        
    def create_model(self):

        model = lstm(neurons=self.neurons, 
                     activation_func = self.activation, 
                     nTimesteps = self.n_timesteps, 
                     nFeatures = self.n_features, 
                     npredTimesteps = 1, 
                     npredFeatures = self.n_predicted_features, 
                     model_optimizer = self.optimizer, 
                     model_loss = self.loss, 
                     model_metrics = self.metrics, 
                     dropout=self.dropout, 
                     conv_layer=self.conv_layer, 
                     nfilters=self.nfilters, 
                     cact =self.conv_act, 
                     cpool=2,
                     classify=self.classification)
        return model
    

    def fit(self, X, y):
        if self.classification:
            self.classes_ = np.unique(y)
        # print(f'shape of Y: {Y.shape}')
        self.model = self.create_model()
        self.model.fit(X, y, nEpochs = 1, nBatches= 64, val_split = 0.2, verb = 2, shuf = False)

        return self

    def predict(self, X):
        # Make predictions
        # print(f'shape of x (predict): {X.shape}')
        y_pred = self.model.predict(X)
        # print(f'shape of y_pred: {y_pred.shape}')
        y_pred = (y_pred >= 0.5).astype(float)
        # print(y_pred[0][0])
        return y_pred 

In [7]:
def custom_accuracy(y_true, y_pred):
    # print(f"Shape of y_true in custom scoring function: {y_true.shape}")
    # print(y_true[0][0])
    # print(f"Shape of y_pred in custom scoring function: {y_pred.shape}")
    # print(y_pred[0][0])

    # # flatten
    y_true_2d = y_true.reshape(-1, y_true.shape[2])  # Shape becomes (2148, 9)
    y_pred_2d = y_pred.reshape(-1, y_pred.shape[2])  # Shape becomes (2148, 9)

    y_true_2d = y_true_2d.astype(int)
    y_pred_2d = y_pred_2d.astype(int)

    # print(f'shape of true and pred after reshape: {y_true_2d.shape} and {y_pred_2d.shape}')
    # print(f'type: {type(y_true_2d[0][0])}')
    # print(f"Data type of y_true: {y_true_2d.dtype}")
    # print(f"Data type of y_pred: {y_pred_2d.dtype}")


    correct = np.sum(y_true == y_pred)
    accuracy = correct / y_true.size


    # Compute accuracy or any custom metric
    return accuracy

In [8]:
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier  # Example model
from scipy.stats import randint, uniform  # For defining distributions
from sklearn.metrics import make_scorer, accuracy_score, f1_score

tscv = TimeSeriesSplit(n_splits = 5)

: 

### 25 npast_timesteps
- Timesteps tried are defined outside of the RandomizedSearch to make preprocessing easier and avoid nan scoring values
- Tried to run 25, 50, 75 timesteps using for loop and list, but kernal died or randomized search terminated due to memory

In [ ]:
past_timepoints = [25, 50, 75]
conv_layer = [False, True]

best_params = []
best_score = []

for i in past_timepoints:
    # reset x and y
    X = np.array(data[numerical].drop(columns=['Time']))
    Y = np.array(data[categorical])

    # preprocess data
    scalerX, X_norm = utils.scale(X)
    scalerY, Y_norm = utils.scale(Y)
    # print(f'x_shape scaled: {X_norm.shape}')
    # print(f'y_shape scaled: {Y_norm.shape}')

    # Reshape based on timesteps
    [X, _] = utils.prep_lstm_data(X_norm, i, 1)
    [_, Y] = utils.prep_lstm_data(Y_norm, i, 1)

    for conv in conv_layer: 
        """will execute with conv_layer False, then conv_layer True, 
        # must be in this order to prevent input shape error
        # The data will reset to 2D when new npast_timesteps in the outter for loop"""
        
        if conv and X.ndim != 4: # reshape input
            X = X.reshape((X.shape[0], 1, X.shape[1], X.shape[2]))
            
        # define parameters to gridsearch
        param_dict = {
        'neurons': [[32, 32], [64, 32], [128, 64, 32]],
        'activation':["tanh", 'relu'],
        'n_timesteps':[i],
        'n_features':[7],
        'n_predicted_timesteps': [1],
        'optimizer': ["adam"],
        'loss': ["binary_crossentropy"],
        'metrics': [["accuracy"]],
        'dropout':[0.1, 0.2, 0.3],
        'conv_layer':[conv],
        'nfilters':[32, 64], 
        'conv_act': ['relu', 'tanh'],
        'pool_size':[2],
        'classification': [True],
        'epochs': [30, 50],
        'batch':[32,64],
        }


        # print(Y.shape)
        # print(type(Y[0][0][0]))


        # print(Y.shape)
        # print(type(Y[0][0][0]))

        # Random Search
        random_search = RandomizedSearchCV(
            estimator= PreprocessingWrapper(),
            param_distributions=param_dict,
            n_iter = 20,  
            scoring = make_scorer(custom_accuracy), 
            cv=tscv,  
            verbose=2,  
            n_jobs= 1,  
            random_state=42  # For reproducibility
        )

        random_search.fit(X, Y)
        best_params.append(random_search.best_params_)
        best_score.append(random_search.best_score_)

    


2024-12-12 13:39:50.391238: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
9/9 - 3s - 302ms/step - accuracy: 0.0017 - loss: 0.6433 - val_accuracy: 0.0000e+00 - val_loss: 0.5828
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=25, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   3.5s
18/18 - 3s - 143ms/step - accuracy: 0.1039 - loss: 0.5740 - val_accuracy: 0.0592 - val_loss: 0.5010
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=25, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   3.2s
27/27 - 3s - 122ms/step - accuracy: 0.2031 - loss: 0.5226 - val_

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


54/54 - 5s - 99ms/step - accuracy: 0.2905 - loss: 0.4025 - val_accuracy: 0.9674 - val_loss: 0.2173
Fitting 5 folds for each of 20 candidates, totalling 100 fits
9/9 - 3s - 378ms/step - accuracy: 0.0000e+00 - loss: 0.6670 - val_accuracy: 0.0000e+00 - val_loss: 0.6143
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=25, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   4.3s
18/18 - 4s - 199ms/step - accuracy: 0.0017 - loss: 0.6097 - val_accuracy: 0.0000e+00 - val_loss: 0.5357
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=25, neurons=[64, 32], nfilters=64, optimiz

In [9]:
# # take the best score and find associated parameters

max_index = best_score.index(max(best_score))
print(f'Best Params for 25 npast_timesteps: {best_params[max_index]}')
print(f'Best Score for 25 npast_timesteps: {max(best_score)}')

Best Params for 25 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 25, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.1, 'conv_layer': False, 'conv_act': 'relu', 'classification': True, 'batch': 64, 'activation': 'tanh'}
Best Score for 25 npast_timesteps: 0.8999931029726188


In [25]:
# append to list
best_scores_ = []
best_params_ = []

best_scores_.append(max(best_score))
best_params_.append(best_params[max_index])


### 50 npast_timesteps

In [9]:
past_timepoints = [50]
conv_layer = [False, True]

best_params_50 = []
best_score_50 = []

for i in past_timepoints:
    # reset x and y
    X = np.array(data[numerical].drop(columns=['Time']))
    Y = np.array(data[categorical])

    # preprocess data
    scalerX, X_norm = utils.scale(X)
    scalerY, Y_norm = utils.scale(Y)
    # print(f'x_shape scaled: {X_norm.shape}')
    # print(f'y_shape scaled: {Y_norm.shape}')

    # Reshape based on timesteps
    [X, _] = utils.prep_lstm_data(X_norm, i, 1)
    [_, Y] = utils.prep_lstm_data(Y_norm, i, 1)

    for conv in conv_layer: 
        """will execute with conv_layer False, then conv_layer True, 
        # must be in this order to prevent input shape error
        # The data will reset to 2D when new npast_timesteps in the outter for loop"""
        
        if conv and X.ndim != 4: # reshape input
            X = X.reshape((X.shape[0], 1, X.shape[1], X.shape[2]))
            
        # define parameters to gridsearch
        param_dict = {
        'neurons': [[32, 32], [64, 32], [128, 64, 32]],
        'activation':["tanh", 'relu'],
        'n_timesteps':[i],
        'n_features':[7],
        'n_predicted_timesteps': [1],
        'optimizer': ["adam"],
        'loss': ["binary_crossentropy"],
        'metrics': [["accuracy"]],
        'dropout':[0.1, 0.2, 0.3],
        'conv_layer':[conv],
        'nfilters':[32, 64], 
        'conv_act': ['relu', 'tanh'],
        'pool_size':[2],
        'classification': [True],
        'epochs': [30, 50],
        'batch':[32,64],
        }


        # print(Y.shape)
        # print(type(Y[0][0][0]))


        # print(Y.shape)
        # print(type(Y[0][0][0]))

        # Random Search
        random_search = RandomizedSearchCV(
            estimator= PreprocessingWrapper(),
            param_distributions=param_dict,
            n_iter=20,  # Number of different combinations to sample
            scoring = make_scorer(custom_accuracy), 
            cv=tscv,  # Number of cross-validation folds
            verbose=2,  # Display search progress
            n_jobs= 1,  # Use all available CPUs
            random_state=42  # For reproducibility
        )

        random_search.fit(X, Y)
        best_params_50.append(random_search.best_params_)
        best_score_50.append(random_search.best_score_)

    


Fitting 3 folds for each of 20 candidates, totalling 60 fits


2024-12-12 11:40:27.265088: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


14/14 - 3s - 201ms/step - accuracy: 0.0152 - loss: 0.5844 - val_accuracy: 0.0000e+00 - val_loss: 0.4806
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=50, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   3.8s
27/27 - 4s - 166ms/step - accuracy: 0.0772 - loss: 0.5252 - val_accuracy: 0.9696 - val_loss: 0.4026
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=50, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   5.7s
41/41 - 6s - 134ms/step - accuracy: 0.4499 - loss: 0.4265 - val_accuracy: 0.9438 - val_loss: 0.3607
34/34 ━━━━━━━━━━━━━━━━━━

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


54/54 - 18s - 324ms/step - accuracy: 0.5451 - loss: 0.4980 - val_accuracy: 0.0000e+00 - val_loss: 0.4279
Fitting 3 folds for each of 20 candidates, totalling 60 fits
14/14 - 4s - 287ms/step - accuracy: 0.5993 - loss: 0.6167 - val_accuracy: 0.9626 - val_loss: 0.5446
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=50, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   5.2s
27/27 - 6s - 224ms/step - accuracy: 0.1217 - loss: 0.5344 - val_accuracy: 0.0000e+00 - val_loss: 0.4179
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=50, neurons=[64, 32], nfilters=64, optimize

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


54/54 - 6s - 114ms/step - accuracy: 0.6915 - loss: 0.5315 - val_accuracy: 0.9673 - val_loss: 0.3426


In [10]:
# take the best score and find associated parameters

max_index_50 = best_score_50.index(max(best_score_50))
print(f'Best Params for 50 npast_timesteps: {best_params_50[max_index_50]}')
print(f'Best Scorefor 50 npast_timesteps: {max(best_score_50)}')

Best Params for 50 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 32, 'neurons': [128, 64, 32], 'n_timesteps': 50, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.1, 'conv_layer': False, 'conv_act': 'tanh', 'classification': True, 'batch': 64, 'activation': 'relu'}
Best Scorefor 50 npast_timesteps: 0.8993370127390746


In [ ]:
best_scores_.append(max(best_score_50))
best_params_.append(best_params_50[max_index_50])

### 75 npast_timesteps

In [9]:
past_timepoints = [75]
conv_layer = [False, True]

best_params_75 = []
best_score_75 = []

for i in past_timepoints:
    # reset x and y
    X = np.array(data[numerical].drop(columns=['Time']))
    Y = np.array(data[categorical])

    # preprocess data
    scalerX, X_norm = utils.scale(X)
    scalerY, Y_norm = utils.scale(Y)
    # print(f'x_shape scaled: {X_norm.shape}')
    # print(f'y_shape scaled: {Y_norm.shape}')

    # Reshape based on timesteps
    [X, _] = utils.prep_lstm_data(X_norm, i, 1)
    [_, Y] = utils.prep_lstm_data(Y_norm, i, 1)

    for conv in conv_layer: 
        """will execute with conv_layer False, then conv_layer True, 
        # must be in this order to prevent input shape error
        # The data will reset to 2D when new npast_timesteps in the outter for loop"""
        
        if conv and X.ndim != 4: # reshape input
            X = X.reshape((X.shape[0], 1, X.shape[1], X.shape[2]))
            
        # define parameters to gridsearch
        param_dict = {
        'neurons': [[32, 32], [64, 32], [128, 64, 32]],
        'activation':["tanh", 'relu'],
        'n_timesteps':[i],
        'n_features':[7],
        'n_predicted_timesteps': [1],
        'optimizer': ["adam"],
        'loss': ["binary_crossentropy"],
        'metrics': [["accuracy"]],
        'dropout':[0.1, 0.2, 0.3],
        'conv_layer':[conv],
        'nfilters':[32, 64], 
        'conv_act': ['relu', 'tanh'],
        'pool_size':[2],
        'classification': [True],
        'epochs': [30, 50],
        'batch':[32,64],
        }


        # print(Y.shape)
        # print(type(Y[0][0][0]))


        # print(Y.shape)
        # print(type(Y[0][0][0]))

        # Random Search
        random_search = RandomizedSearchCV(
            estimator= PreprocessingWrapper(),
            param_distributions=param_dict,
            n_iter=20,  # Number of different combinations to sample
            scoring = make_scorer(custom_accuracy), 
            cv=tscv,  # Number of cross-validation folds
            verbose=2,  # Display search progress
            n_jobs= 1,  # Use all available CPUs
            random_state=42  # For reproducibility
        )

        random_search.fit(X, Y)
        best_params_75.append(random_search.best_params_)
        best_score_75.append(random_search.best_score_)

    


Fitting 3 folds for each of 20 candidates, totalling 60 fits


2024-12-12 11:55:28.332335: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


14/14 - 6s - 423ms/step - accuracy: 0.6694 - loss: 0.5869 - val_accuracy: 0.9624 - val_loss: 0.4511
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   8.2s
27/27 - 7s - 269ms/step - accuracy: 0.6492 - loss: 0.5025 - val_accuracy: 0.9694 - val_loss: 0.3635
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=False, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   9.1s
40/40 - 9s - 230ms/step - accuracy: 0.6064 - loss: 0.4332 - val_accuracy: 0.9498 - val_loss: 0.3569
34/34 ━━━━━━━━━━━━━━━━━━━━ 1

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


54/54 - 17s - 317ms/step - accuracy: 0.2356 - loss: 0.6013 - val_accuracy: 0.9671 - val_loss: 0.5133
Fitting 3 folds for each of 20 candidates, totalling 60 fits
14/14 - 3s - 224ms/step - accuracy: 0.7753 - loss: 0.6218 - val_accuracy: 0.9624 - val_loss: 0.5435
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, pool_size=2; total time=   4.0s
27/27 - 5s - 185ms/step - accuracy: 0.5921 - loss: 0.4843 - val_accuracy: 0.9694 - val_loss: 0.3808
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
[CV] END activation=tanh, batch=32, classification=True, conv_act=tanh, conv_layer=True, dropout=0.1, epochs=50, loss=binary_crossentropy, metrics=['accuracy'], n_features=7, n_predicted_timesteps=1, n_timesteps=75, neurons=[64, 32], nfilters=64, optimizer=adam, 

/home/kxlee/miniconda3/envs/msse-python/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


54/54 - 5s - 97ms/step - accuracy: 0.1428 - loss: 0.4902 - val_accuracy: 0.9671 - val_loss: 0.2961


In [10]:
# take the best score and find associated parameters

max_index_75 = best_score_75.index(max(best_score_75))
print(f'Best Params for 75 npast_timesteps: {best_params_75[max_index_75]}')
print(f'Best Scorefor 75 npast_timesteps: {max(best_score_75)}')

Best Params for 75 npast_timesteps: {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 32, 'neurons': [128, 64, 32], 'n_timesteps': 75, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.3, 'conv_layer': False, 'conv_act': 'relu', 'classification': True, 'batch': 32, 'activation': 'relu'}
Best Scorefor 75 npast_timesteps: 0.9011414807833281


In [16]:
best_scores_.append(max(best_score_75))
best_params_.append(best_params_75[max_index_75])

### Best Parameters for Classification

In [20]:
max_index_all = best_scores_.index(max(best_scores_))
best_params_all = best_params_[max_index_all]

print(f'Best Params for Classification: {best_params_all}')
print(f'Best Score for Classification: {max(best_scores_)}')

Add manually to list becuase kernal crashed, this was test run with 10 iterations, cv = 2

In [11]:
scores = [0.8999931029726188, 0.8993370127390746,  0.9011414807833281]
params = [{'pool_size': 2, 'optimizer': 'adam', 'nfilters': 64, 'neurons': [64, 32], 'n_timesteps': 25, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.1, 'conv_layer': False, 'conv_act': 'relu', 'classification': True, 'batch': 64, 'activation': 'tanh'},
          {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 32, 'neurons': [128, 64, 32], 'n_timesteps': 50, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.1, 'conv_layer': False, 'conv_act': 'tanh', 'classification': True, 'batch': 64, 'activation': 'relu'},
          {'pool_size': 2, 'optimizer': 'adam', 'nfilters': 32, 'neurons': [128, 64, 32], 'n_timesteps': 75, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.3, 'conv_layer': False, 'conv_act': 'relu', 'classification': True, 'batch': 32, 'activation': 'relu'}


]
max_index = scores.index(max(scores))
print(max(scores))
print(max_index)
print(params[max_index])


0.9011414807833281
2
{'pool_size': 2, 'optimizer': 'adam', 'nfilters': 32, 'neurons': [128, 64, 32], 'n_timesteps': 75, 'n_predicted_timesteps': 1, 'n_features': 7, 'metrics': ['accuracy'], 'loss': 'binary_crossentropy', 'epochs': 30, 'dropout': 0.3, 'conv_layer': False, 'conv_act': 'relu', 'classification': True, 'batch': 32, 'activation': 'relu'}
